If any problems occur, please refer to the E2_Vector_Search to check out the set up process. 

In [ ]:
# Import Vertex AI SDK
PROJECT_ID = !gcloud config get project
PROJECT_ID = PROJECT_ID.n
LOCATION = "europe-west2"
LOCATION_DEPLOY = "europe-west2" #Location to deploy GCP resources

import vertexai
from google.cloud import aiplatform

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [33]:
from vertexai.preview.language_models import ChatModel

class PaLMWrapper:
    def __init__(self):
        self.chat_model = ChatModel.from_pretrained("chat-bison@001")
        self.parameters = {
            "temperature": 0.3,
            "max_output_tokens": 256,
            "top_p": 0.95,
        }
    
    def generate_response(self, context, message):
        chat = self.chat_model.start_chat(context=context)
        response = chat.send_message(message, **self.parameters)
        return response.text

In [2]:
pw = PaLMWrapper()
response = pw.generate_response(context='you are a london weather forecaster', message='whats the weather like today')
response

"Today's weather in London is cloudy with a high of 15 degrees Celsius and a low of 10 degrees Celsius. There is a 20% chance of rain."

In [3]:
import pandas as pd

import vertexai
from vertexai.preview.language_models import TextEmbeddingModel
from google.cloud import aiplatform

class VertexAIVectorStore:
    def __init__(self):
        
        # god awful way of doing it, should be a config and passed through but oh well hacky hack
        self.gen_ai_index_endpoint = aiplatform.MatchingEngineIndexEndpoint(
            index_endpoint_name="projects/playpen-8d8611/locations/europe-west2/indexEndpoints/5891412000042385408"
        )

        self.gen_ai_index = aiplatform.MatchingEngineIndex(
            index_name="projects/playpen-8d8611/locations/europe-west2/indexes/2680908415680643072"
        )
        
        self.model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")
        self.df = pd.read_csv('text_data_g_embedding.csv')
        
    def search(self, input, k=3):
        embedding_vec =  self.model.get_embeddings([input])[0].values #Send request to embedding model to generate the embedding vector

        #find neighbours using vector search
        neighbours = self.gen_ai_index_endpoint.find_neighbors(
            deployed_index_id="gen_ai_deployed_index",
            queries=[embedding_vec],
            num_neighbors=k,
        )[0]
        
        results = []
        for nb in neighbours:
            text_id = nb.id
            text = self.df.iloc[int(nb.id)]['text']
            score = nb.distance
            results.append((text_id, text, score))
        
        return results


In [4]:
vai = VertexAIVectorStore()
vector = vai.search(input='what is credit card fraud')
vector

[('41',
  'Security & fraudLost or stolen cardsReport a lost or stolen Business CardReport a lost or stolen Business payment cardReport a lost or stolen Corporate CardReport a lost or stolen Corpoarate cardReport a fraudReport a fraud on our business accountsMake a complaintReport fraudulent use of online bankingStaying safeProtect your business from fraudManage the cyber threat to your business',
  0.7093294858932495),
 ('55',
  "Report\xa0fraud on online business bankingCall us immediately\xa0 if you think any of your Lloyds Business Bank accounts have been:accessed by a third party\xa0ortargeted for\xa0fraud.Please get in touch even if you haven't suffered any loss from the attempted fraud.This page is for business customersFind out how to report fraud if you’re a personal customerReport FraudBy PhoneOnline for Business\xa00800 056 3099Commercial Banking Online\xa00800 169 1296+44 1293 762 380 from abroad.\xa0Lines are open 24 hours.You'll need to tell us the reason for your call so

In [42]:
# couldnt get langchain to work so custom RAG anyone?

class RAG:
    def __init__(self, vector_store, palm_wrapper, initial_system_prompt=True):
        self.vector_store = vector_store
        self.palm_wrapper = palm_wrapper
        # the comment from mathew regarding memory
        self.conversation_history = []
        
        if initial_system_prompt:            
            system_prompt = r"You are a professional assistant with extensive experience helping numerous small and medium businesses. You work for a large retail bank called Lloyds. Please assist the user answering questions with detailed responses, providing reasoning whenever prescriptive advice is given."
            self.conversation_history.append((system_prompt, ""))
        
    
    def generate(self, query, is_user_query=True, k=3):
        
        contexts=[]
        sources=[]
        scores=[]
        
        if is_user_query:
            # if user query, perform vector db search
            # retrieve contexts based on the query
            search_results = self.vector_store.search(query, k)
            contexts = [result[1] for result in search_results]
            sources = [result[0] for result in search_results]
            scores = [result[2] for result in search_results]
            
        # combine both query and response from conversation history
        history_context = '\n'.join(['Q: ' + query + '\nA: ' + response for query, response in self.conversation_history[-k:]])
        
        # combine history context and current contexts
        combined_context = '\n'.join([history_context] + contexts)
        response = self.palm_wrapper.generate_response(combined_context[:20000], query)
        
        # update conversation history with current interaction
        self.conversation_history.append((query, response))
        
        
        return response, sources, scores

In [43]:
rag = RAG(vai, pw)
query = "how do i change my address?"
response, sources, scores = rag.generate(query)
print(response)
print('sources used:', sources)
print('scores:', scores)

You can change your business address by completing a form and sending it to us. You can do this online if you're a full access user, or you can download a form and send it to us by post.
sources used: ['24', '26', '45']
scores: [0.7175062894821167, 0.6619007587432861, 0.6554778814315796]


In [44]:
rag = RAG(vai, pw)
query = "i've lost my card"
response, sources, scores = rag.generate(query)
print(response)
print('sources used:', sources)
print('scores:', scores)

query = "why?"
response, sources, scores = rag.generate(query)
print(response)
print('sources used:', sources)
print('scores:', scores)

query = "i was not asking about an audit request"
response, sources, scores = rag.generate(query)
print(response)
print('sources used:', sources)
print('scores:', scores)

query = "what is the weather in london"
response, sources, scores = rag.generate(query)
print(response)
print('sources used:', sources)
print('scores:', scores)

query = "how can i set up an online banking account?"
response, sources, scores = rag.generate(query)
print(response)
print('sources used:', sources)
print('scores:', scores)

I'm sorry to hear that you've lost your card. It's important that you report this to us immediately so that we can cancel the card and prevent any fraudulent activity. You can do this by calling us on 0800 096 9779. If you're outside the UK, you can call +44 1702 278 270.
sources used: ['56', '53', '62']
scores: [0.7142672538757324, 0.7047110795974731, 0.7031532526016235]
We need a letter of authority before we can process an audit request. It must be on your headed paper and include details of all the legal entities you are happy for the bank to disclose information to your auditors for.
sources used: ['50', '51', '16']
scores: [0.6711143851280212, 0.6682686805725098, 0.6521599292755127]
I'm sorry, I misunderstood. What can I help you with today?
sources used: ['50', '16', '67']
scores: [0.6632628440856934, 0.6557632684707642, 0.6330248713493347]
I'm sorry, I can't help you with that. I'm an AI-powered chatbot that can help you with your banking needs. I can't help you with general in

In [29]:
import ipywidgets as widgets
from IPython.display import display, clear_output

class ChatUI:
    def __init__(self, rag_instance):
        self.rag_instance = rag_instance
        self.conversation = []
        self._setup_ui()
    
    def _setup_ui(self):
        self.input_box = widgets.Text(
            placeholder='Message LBG help and support...',
            description='Lloyds',
            layout={'width': '80%'}
        )
        
        self.send_button = widgets.Button(
            description='Send',
            button_style='info',
            layout={'width': '12%'}
        )
        self.output_area = widgets.Output(layout={'border': '1px solid black', 'width': '100%'})
        self.send_button.on_click(self._on_send_clicked)
        
        input_send_box = widgets.HBox([self.input_box, self.send_button])
        
        display(self.output_area, input_send_box)
        
    def _on_send_clicked(self, b):
        query = self.input_box.value
        self.conversation.append(f"You: {query}")
        response = self.rag_instance.generate(query)
        self.conversation.append(f'LBG AI: {response}')
        
        with self.output_area:
            clear_output(wait=True)
            print('\n'.join(self.conversation))
        
        # clear input box
        self.input_box.value = ''
        
rag = RAG(vai, pw)
chat_ui = ChatUI(rag)            

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

The below didn't work for some reason, logging off for now

In [47]:
# Hide all cells
from IPython.display import display, Javascript

def hide_other_cells():
    script = '''
    require(["base/js/namespace"], function(Jupyter) {
        Jupyter.notebook.get_cells().forEach(function(cell) {
            if (cell.cell_type === 'code' && !cell.output_area.outputs.length) {
                cell.element.hide(); //Hides all cells that don't have an output
            }
        });
    });
    '''
    
    display(HTML(javascript))

hide_other_cells()